In [20]:
import pandas as pd
import matplotlib.pyplot as plt  
import numpy as np 
import seaborn as sns 

In [21]:
data = pd.read_csv('SBAnational.csv')

/var/folders/83/skc59h713n1f5fpwznbhnkr80000gn/T/ipykernel_4891/1533426435.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('SBAnational.csv')


##### On crée un nouveau dataframe où les colonnes qui sont inutiles pour les prédictions sont supprimées ainsi que les variables suspectées de faire du dataleaking.

In [22]:
data_lite = data.drop(['ApprovalDate', 'City', 'ApprovalFY', 'BalanceGross', 'NoEmp', 'LoanNr_ChkDgt', 'Name', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross', 'BankState', 'State', 'ChgOffPrinGr', 'GrAppv'], axis=1)
data_lite.head(5)

,Zip,Bank,NAICS,Term,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status,SBA_Appv
0,47711,FIFTH THIRD BANK,451120,84,2.0,0,0,1,0,N,Y,P I F,"$48,000.00"
1,46526,1ST SOURCE BANK,722410,60,2.0,0,0,1,0,N,Y,P I F,"$32,000.00"
2,47401,GRANT COUNTY STATE BANK,621210,180,1.0,0,0,1,0,N,N,P I F,"$215,250.00"
3,74012,1ST NATL BK & TR CO OF BROKEN,0,60,1.0,0,0,1,0,N,Y,P I F,"$28,000.00"
4,32801,FLORIDA BUS. DEVEL CORP,0,240,1.0,7,7,1,0,N,N,P I F,"$229,000.00"


#####  Vérifier les types des variables, les valeurs manquantes ou suspectes.
On vérifie les valeurs uniques pour chaque variable du dataframe ainsi que les 'missing values'.

In [23]:
print(data_lite.isnull().sum())

data_lite.dtypes


Zip                 0
Bank             1559
NAICS               0
Term                0
NewExist          136
CreateJob           0
RetainedJob         0
FranchiseCode       0
UrbanRural          0
RevLineCr        4528
LowDoc           2582
MIS_Status       1997
SBA_Appv            0
dtype: int64


Zip                int64
Bank              object
NAICS              int64
Term               int64
NewExist         float64
CreateJob          int64
RetainedJob        int64
FranchiseCode      int64
UrbanRural         int64
RevLineCr         object
LowDoc            object
MIS_Status        object
SBA_Appv          object
dtype: object

Les variables numériques: NAICS, Term, NewExist, FranchiseCode, UrbanRural

Les variables catégorielles: ApprovalDate, ApprovalFY, RevLineCr, LowDoc, MIS_Status (la target).

In [24]:
"""#pour la colonne ApprovalDate, on va réduire les modalités en ne considérant que l'année
# d'abord transformer la date en Y-M-D

data_lite['ApprovalDate'] = pd.to_datetime(data_lite['ApprovalDate'], format='%d-%b-%y', errors='coerce')
data_lite['ApprovalDate_Year'] = data_lite['ApprovalDate'].dt.year
data_lite = data_lite.drop('ApprovalDate', axis=1)

#On verifie que les colonnes 'ApprovalDateYear' et 'ApprovalFY' sont identiques
data_lite['ApprovalDate_Year'].equals(data_lite['ApprovalFY'])
data_lite.head(2)"""

"#pour la colonne ApprovalDate, on va réduire les modalités en ne considérant que l'année\n# d'abord transformer la date en Y-M-D\n\ndata_lite['ApprovalDate'] = pd.to_datetime(data_lite['ApprovalDate'], format='%d-%b-%y', errors='coerce')\ndata_lite['ApprovalDate_Year'] = data_lite['ApprovalDate'].dt.year\ndata_lite = data_lite.drop('ApprovalDate', axis=1)\n\n#On verifie que les colonnes 'ApprovalDateYear' et 'ApprovalFY' sont identiques\ndata_lite['ApprovalDate_Year'].equals(data_lite['ApprovalFY'])\ndata_lite.head(2)"

Vérifier les valeurs uniques des colonnes. Si les valeurs différent des modalités officielles, on les remplace par NaN.

In [25]:
data_lite['MIS_Status'].unique() 

# on supprime les valeurs manquantes de la target
data_lite.dropna(subset=['MIS_Status'], inplace=True)

# encodage de la target
data_lite['MIS_Status'] = data_lite['MIS_Status'].map({'P I F': 0, 'CHGOFF': 1})
data_lite.MIS_Status.dtype



dtype('int64')

In [26]:
data_lite['RevLineCr'].unique() # renvoie:array(['N', '0', 'Y', 'T',......, 'Q']

valeurs_odd = [ 'NaN', 'nan','0', 'T','`', ',', '1', 'C', '3', '2', 'R', '7','A', '5', '.', '4', '-', 'Q']

data_lite['RevLineCr'] = data_lite['RevLineCr'].replace(valeurs_odd, np.nan)

data_lite['RevLineCr'].unique()

array(['N', nan, 'Y'], dtype=object)

In [27]:
data_lite['Term'].unique() #valeurs numériques 

array([ 84,  60, 180, 240, 120,  45, 297, 162,  12, 300,  87, 114, 144,
       126,  83, 102,  80, 137,  42,  96, 167,   7,  36,  37,  26, 264,
        72,  24,   5,  54,  66, 161,  71,   4,  93, 288, 108,  10,  13,
        90,  19,  16,   3,  27, 149,  41, 246,  18,  57, 104,  82, 298,
        14,  61, 127,  58,  44,  32,  85,  48,  31, 112,  38,  73,  47,
        11, 134,  15,  79,  53,  39,   6, 255,  55, 133,  95,  35,  59,
        62,  68, 123,  46,  70, 138,  40,  52,  25,  65,  91,   1,  74,
        49, 103,  77,  86,  63,  56,  22,   0,  97,  23,  17,  69,  21,
        43,  89, 276,  92, 183,   2, 132,  34, 131,   9,  78,  99, 129,
       216,   8,  29, 289,  30, 119, 228, 168, 208,  81, 147, 125,  94,
        51, 211,  64, 111, 266,  75, 306,  28, 232, 117, 118, 309, 303,
        98, 191, 116,  76, 113, 292,  88, 166, 244, 176, 258, 203, 231,
       142,  33, 157, 165,  50, 210, 294, 301, 106,  20, 318, 229, 204,
       269, 241, 178, 115, 174, 192,  67, 100, 141, 282, 122, 15

In [28]:
data_lite['NewExist'].unique()

#relever les lignes avec la valeur 0 de ma colonne
rows_with_zero = data_lite.loc[data_lite['NewExist']==0]
rows_with_zero

#remplacer les 0 par des Nan
data_lite['NewExist'] = data_lite['NewExist'].replace(0, np.nan)
data_lite['NewExist'].unique()

array([ 2.,  1., nan])

In [29]:
data_lite['FranchiseCode'].unique()

array([    1,     0, 15100, ...,  2899, 18701, 15930])

In [30]:
data_lite['UrbanRural'].unique() #remplace 0 par NaN

#remplacer les 0 par des Nan
data_lite['UrbanRural'] = data_lite['UrbanRural'].replace(0, np.nan)
data_lite['UrbanRural'].unique()


array([nan,  1.,  2.])

In [31]:
data_lite['LowDoc'].unique()
valeurs_odd = ['C', '1','S', 'R', 'A', '0']
data_lite['LowDoc'] = data_lite['RevLineCr'].replace(valeurs_odd, np.nan)
data_lite['LowDoc'].unique()

array(['N', nan, 'Y'], dtype=object)

In [32]:
data_lite.dtypes

Zip                int64
Bank              object
NAICS              int64
Term               int64
NewExist         float64
CreateJob          int64
RetainedJob        int64
FranchiseCode      int64
UrbanRural       float64
RevLineCr         object
LowDoc            object
MIS_Status         int64
SBA_Appv          object
dtype: object

In [33]:
#enlever le symbole $ de sbvappv

data_lite["SBA_Appv"] = data_lite["SBA_Appv"].str.replace("$", "")
data_lite.head(2)
data_lite.dtypes

Zip                int64
Bank              object
NAICS              int64
Term               int64
NewExist         float64
CreateJob          int64
RetainedJob        int64
FranchiseCode      int64
UrbanRural       float64
RevLineCr         object
LowDoc            object
MIS_Status         int64
SBA_Appv          object
dtype: object

In [34]:
#la franchise code assigne un code lorsque l'entreprise est une franchise, ou 1 ou 0 lorsque ce n'est pas une franchise
# on peut remplacer les valeurs par '0': pas de franchise et '1': franchise en 2 étapes

data_lite.loc[data_lite['FranchiseCode']  <=1, 'FranchiseCode'] = 0

data_lite.loc[data_lite['FranchiseCode'] > 0, 'FranchiseCode'] = 1

data_lite.tail(2)

,Zip,Bank,NAICS,Term,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status,SBA_Appv
899162,96830,BANK OF HAWAII,0,60,1.0,0,0,0,NaN,N,N,1,"60,000.00"
899163,96734,CENTRAL PACIFIC BANK,0,48,2.0,0,0,0,NaN,N,N,0,"24,000.00"


In [35]:
#supprimer les ',' dans la colonne SBA_Appv
data_lite['SBA_Appv'] = data_lite['SBA_Appv'].str.replace(',', '').str.strip()
data_lite['SBA_Appv'] = data_lite['SBA_Appv'].astype(float)
data_lite['SBA_Appv'].dtypes

dtype('float64')

#### Enregistre le dataframe data_lite pour le réutiliser dans le notebook modelisation


In [36]:
data_lite.to_csv('data.csv', index=False)

